In [6]:
import torch
from src.myChatGPT import *
from src.metrics import *
#from src.exp_weighted_pairs import *

import matplotlib.pyplot as plt
import numpy as np

In [210]:
with open("data/input.txt", "r", encoding = "utf-8") as f:
    text=f.read()





In [119]:
def generate_bigrams(vocab_size, data):
    return torch.concatenate([data[:-1].unsqueeze(1),data[1:].unsqueeze(1)], dim=1) 

def generate_bigrams_id(vocab_size, data):
    return generate_bigrams(vocab_size, data) @ torch.tensor([1,vocab_size])

def count_sequence_of_bigrams(vocab_size, data):
    a = generate_bigrams_id(vocab_size, data)
    return torch.zeros(vocab_size*vocab_size, dtype=torch.long).scatter_add(0, a, torch.ones_like(a))

In [211]:
class Tokenizer:
    def __init__(self, text):
        self.vocab = list(set(text))
        self.vocab.sort()

        self.ctoi = { self.vocab[i]:i for i in range(len(self.vocab)) }
        self.itoc = { i:self.vocab[i] for i in range(len(self.vocab)) }


    def get_vocab_size(self):
        return len(self.vocab)

    def encode(self, s): 
        data = torch.tensor([ self.ctoi[i] for i in s ], dtype=torch.long)
        return data

    def decode(self, t): 
        return "".join([ self.itoc[i] for i in t ])



In [222]:
def bpe(data, vocab):
    
    vocab_size = len(vocab)

    bigrams = torch.concatenate([data[:-1].unsqueeze(1),data[1:].unsqueeze(1)], dim=1)
    bigrams_id = bigrams @ torch.tensor([1,vocab_size])
    bigrams_count = torch.zeros(vocab_size*vocab_size, dtype=torch.long).scatter_add(0, bigrams_id, torch.ones_like(bigrams_id))

    most_popular_bigram = bigrams_count.argmax()

    pos = ((bigrams_id == most_popular_bigram).nonzero(as_tuple=True)[0])

    mask = data>=0
    mask[pos] = False
    data[pos+1] = vocab_size
    p = (vocab[most_popular_bigram % vocab_size], vocab[most_popular_bigram//vocab_size])
    #print(p)
    vocab.append("%s%s" % p)
    return data[mask], vocab

#text = "abcdabab"

vocab = list(set(text))
vocab.sort()

t = Tokenizer(text)
data = t.encode(text)

print(data, vocab)

for i in range(256):
    data, vocab = bpe(data, vocab)
    #print()

print(len(data), vocab)


tensor([18, 47, 56,  ..., 45,  8,  0]) ['\n', ' ', '!', '$', '&', "'", ',', '-', '.', '3', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']
568208 ['\n', ' ', '!', '$', '&', "'", ',', '-', '.', '3', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', 'e ', 'th', 't ', 's ', 'd ', ', ', 'ou', 'er', 'in', 'y ', 'an', ':\n', 'or', 'o ', 'en', '\n\n', 'ar', ' th', 'on', 'll', 'ha', ',\n', '.\n\n', 'is ', 'es', 'you', ' s', 'to ', 'and ', 'ow', 'ea', ' m', ' w', 'of', ' h', 'ing', 'om', ' a', 'ch', 'the ', 'st', ' b', 'no', 'ir', 'for', 've ', 'e, ',

In [ ]:
print("".join([ vocab[i] for i in data ]))

